# odoo中的安全
- odoo中采用两种数据驱动的机制来管理和限制对数据的访问，两种安全机制机制都是与权限组相关联，而用户可以归属于任一权限组，从而将安全机制应用到用户。

## res.groups类
- name：作为用户可读的组标识（指明角色或者该权限组的作用）
- category_id：用户组所属的模块名，用于将组与Odoo App(一组相关的业务模型)关联起来，并将它们转换为用户表单中的独家选择。
- implied_ids：基于哪个用户组，类似伪继承
- users：预设用户属于这个用户组
- comments:备注

## 访问权限：ir.model.access类
- name：这个角色组的作用
- model_id：被控制访问的模型
- group_id：被授予访问权限的权限组
- perm_create:
- perm_read:
- perm_write:
- perm_unlink:

## 访问规则：ir.rule
- record的rule是为了允许操作而必须满足的条件。record的rule按照访问权限逐条记录进行评估
- name：rule的描述
- model_id:规则应用的模型
- groups：授予(或不授予)访问权限的res.groups。可以指定多个组。如果没有指定组，则该规则是全局规则，其处理方式与“组”规则不同(见下文)。
- global：
- domain_force:

# 访问权限
- 